In [21]:
import tensorflow as tf

In [22]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

#### Load and prepare the MNIST dataset

In [23]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

Add a channel dimension

In [24]:
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

use `tf.data` to batch and shuffle the dataset

In [25]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

Build the `tf.keras` model using the Keras model subclassing API:

In [26]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10)

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

model = MyModel()

Choose an optimizer and loss function for training:

In [27]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

Select metrics to measure the loss and the accuracy of the model.These metrcs accumulate the values over epochs and then print the overall result

In [28]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

Use `tf.GradientTape` to train the model:

In [29]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

Test the model:

In [30]:
@tf.function
def test_step(images, labels):
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [31]:
EPOCHS = 5

for epoch in range(EPOCHS):
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )

Epoch 1, Loss: 0.1324303150177002, Accuracy: 96.00166320800781, Test Loss: 0.059119727462530136, Test Accuracy: 97.89999389648438
Epoch 2, Loss: 0.04110975190997124, Accuracy: 98.71500396728516, Test Loss: 0.05593622848391533, Test Accuracy: 98.16999816894531
Epoch 3, Loss: 0.021383393555879593, Accuracy: 99.30500030517578, Test Loss: 0.05904184281826019, Test Accuracy: 98.07999420166016
Epoch 4, Loss: 0.013409532606601715, Accuracy: 99.53333282470703, Test Loss: 0.056397415697574615, Test Accuracy: 98.33999633789062
Epoch 5, Loss: 0.009339621290564537, Accuracy: 99.69166564941406, Test Loss: 0.06659878045320511, Test Accuracy: 98.3499984741211
